# Step 1: Import necessary libraries

In [1]:
import os
import cv2
import numpy as np
import pickle
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Step 2: Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [27]:
datagen

# Step 3: Function to extract faces using MTCNN

In [2]:
detector = MTCNN()

In [ ]:
detector

In [3]:
def extract_face(image, required_size=(160, 160)):
    results = detector.detect_faces(image)
    if len(results) == 0:
        return None
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = image[y1:y2, x1:x2]
    face = cv2.resize(face, required_size)
    return face

# Step 4: Load and augment images

In [33]:
def load_images_from_folder(folder, augment=False):
    images = []
    labels = []
    for person_name in os.listdir(folder):
        person_folder = os.path.join(folder, person_name)
        if not os.path.isdir(person_folder):
            continue
        for filename in os.listdir(person_folder):
            img_path = os.path.join(person_folder, filename)
            img = cv2.imread(img_path)
            face = extract_face(img)
            if face is not None:
                images.append(face)
                labels.append(person_name)
                if augment:
                    face = face.reshape((1, ) + face.shape)
                    for i, batch in enumerate(datagen.flow(face, batch_size=1)):
                        augmented_img = batch[0].astype('uint8')
                        images.append(augmented_img)
                        labels.append(person_name)
                        save_augmented_image(augmented_img, person_name, filename, i)
                        if i >= 4:  # Augment 5 images per original image
                            break
    return np.array(images), np.array(labels)

In [34]:
def save_augmented_image(image, person_name, original_filename, idx):
    output_folder = os.path.join("augmented_data", person_name)
    os.makedirs(output_folder, exist_ok=True)
    base_name = os.path.splitext(original_filename)[0]
    output_path = os.path.join(output_folder, f"{base_name}_aug_{idx}.png")
    cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

# Step 5: Load dataset with augmentation

In [8]:
dataset_path = "data"

In [ ]:
images, labels = load_images_from_folder(dataset_path, augment=True)

In [37]:
images

array([[[[  5,   7,   8],
         [  4,   5,   7],
         [  0,   1,   3],
         ...,
         [  1,   1,   1],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  5,   7,   8],
         [  4,   6,   8],
         [ 10,  11,  15],
         ...,
         [  2,   2,   2],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  1,   2,   3],
         [  1,   2,   4],
         [ 15,  17,  21],
         ...,
         [  2,   2,   2],
         [  0,   0,   0],
         [  0,   0,   0]],

        ...,

        [[  2,   2,   2],
         [  2,   2,   2],
         [  1,   1,   1],
         ...,
         [ 80, 112, 177],
         [ 74, 107, 176],
         [ 67, 102, 175]],

        [[  2,   2,   2],
         [  2,   2,   2],
         [  1,   1,   1],
         ...,
         [ 79, 112, 180],
         [ 76, 110, 180],
         [ 71, 105, 179]],

        [[  2,   2,   2],
         [  2,   2,   2],
         [  2,   2,   2],
         ...,
         [ 75, 108, 182],
        

In [38]:
labels

array(['Cristiano_Ronaldo', 'Cristiano_Ronaldo', 'Cristiano_Ronaldo', ...,
       'Virat_Kohli', 'Virat_Kohli', 'Virat_Kohli'], dtype='<U17')

# Step 6: Encode labels

In [40]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels)

In [41]:
label_encoder

LabelEncoder()

In [42]:
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

# Step 7: Split the dataset

In [44]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [45]:
X_train, X_test, y_train, y_test

(array([[[[ 61,  57,  63],
          [ 61,  57,  63],
          [ 61,  57,  63],
          ...,
          [ 42,  38,  37],
          [ 41,  37,  36],
          [ 40,  36,  35]],
 
         [[ 61,  57,  63],
          [ 61,  57,  63],
          [ 61,  57,  63],
          ...,
          [ 42,  38,  37],
          [ 41,  37,  36],
          [ 40,  36,  35]],
 
         [[ 61,  57,  63],
          [ 61,  57,  63],
          [ 61,  57,  63],
          ...,
          [ 42,  38,  37],
          [ 41,  37,  36],
          [ 40,  36,  35]],
 
         ...,
 
         [[ 35,  30,  31],
          [ 35,  30,  31],
          [ 35,  30,  31],
          ...,
          [110, 103, 129],
          [141, 132, 159],
          [175, 165, 191]],
 
         [[ 35,  30,  31],
          [ 35,  30,  31],
          [ 35,  30,  31],
          ...,
          [150, 138, 165],
          [184, 171, 197],
          [205, 191, 216]],
 
         [[ 35,  30,  31],
          [ 35,  30,  31],
          [ 35,  30,  31],
   

# Step 8: Define the model

In [47]:
model = Sequential([
    Flatten(input_shape=(160, 160, 3)),
    Dense(100, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

E:\Face_Recognition_MTCNN\myenv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [48]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 9: Train the model

In [50]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 117ms/step - accuracy: 0.5007 - loss: 3966.8728 - val_accuracy: 0.7752 - val_loss: 539.0117
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 112ms/step - accuracy: 0.6697 - loss: 543.5287 - val_accuracy: 0.7468 - val_loss: 492.5075
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.7364 - loss: 365.1393 - val_accuracy: 0.5866 - val_loss: 712.1213
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 113ms/step - accuracy: 0.7557 - loss: 285.6666 - val_accuracy: 0.8295 - val_loss: 180.3039
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.7792 - loss: 150.6917 - val_accuracy: 0.7235 - val_loss: 131.3961
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 113ms/step - accuracy: 0.8010 - loss: 122.3339 - val_accuracy: 0.7829 - val_loss: 111.3084
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - accuracy: 0.8528 - loss: 71.9664 - val_accuracy: 0.8243 - val_loss: 116.8973
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 120ms/step - accuracy: 0.8122 - los

In [51]:
model

<Sequential name=sequential, built=True>

# Step 10: Save the model using pickle

In [4]:
model_path = "face_recognition_model.pkl"

In [54]:
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

# Step 11: Load the model using pickle

In [5]:
with open(model_path, 'rb') as f:
    model = pickle.load(f)

In [6]:
# Step 12: Real-time face recognition
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    faces = detector.detect_faces(frame)
    for face in faces:
        x, y, width, height = face['box']
        x, y = abs(x), abs(y)
        x2, y2 = x + width, y + height
        face_crop = frame[y:y2, x:x2]
        face_crop = cv2.resize(face_crop, (160, 160))
        face_crop = face_crop.astype('float32') / 255.0
        face_crop = np.expand_dims(face_crop, axis=0)
        
        # Predict the class
        prediction = model.predict(face_crop)
        class_index = np.argmax(prediction)
        class_name = label_encoder.classes_[class_index]
        
        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, class_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    cv2.imshow('Face Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━